# KPIs

En este apartado realizaremos la creacion de los dataframes necesarios para el calculo de cada KPIs a apartir del dataframe orginal que es el df_siniestros

In [26]:
# Importar librerías
import pandas as pd
import numpy as np

In [27]:
# Se lee el dataset 01-homicidios_total.csv
homicidios = pd.read_csv("data/01-homicidios-total.csv")
poblacion = pd.read_csv("data/02-poblacion-total.csv")

# KPI 1
### Reducir en un 10% la tasa de homicidios en siniestros viales de los últimos seis meses, en CABA, en comparación con la tasa de homicidios en siniestros viales del semestre anterior.

In [28]:
# Se crea una nueva variable "semestre" que divide el año en dos
homicidios["semestre"] = homicidios["mes"].apply(lambda x: 1 if x <= 6 else 2)

### Teniendo la columna `semestres` se realiza la agrupación por Año y Semestre para poder sumar la cantidad de homicidios que corresponde a cada uno

In [31]:
agrupacion_semestre = (
    homicidios.groupby(["anio", "semestre"]).size().reset_index(name="homicidios")
)

In [32]:
# Observamos los primeros registros
agrupacion_semestre.head()

,anio,semestre,homicidios
0,2016,1,65
1,2016,2,81
2,2017,1,69
3,2017,2,71
4,2018,1,70


### Se calcula una nueva variable Homicidios Semestre Anterior donde se calcula para cada semestre anterior la cantidad de homicidios

In [33]:
agrupacion_semestre["semestre_pasado"] = agrupacion_semestre["homicidios"].shift(
    periods=1, fill_value=0
)

In [34]:
# Observamos los primeros registros
agrupacion_semestre.head()

,anio,semestre,homicidios,semestre_pasado
0,2016,1,65,0
1,2016,2,81,65
2,2017,1,69,81
3,2017,2,71,69
4,2018,1,70,71


### Se calcula la tasa de homicidios para cada semestre cada 100000 habitantes

In [35]:
# Seleccionas la población total del año 2021
pob_total_2021 = poblacion[poblacion["Año"] == 2021]["Total"].iloc[0]
pob_total_2021

3078836.0

In [36]:
# Se calcula la tasa de homicidios por semestre
agrupacion_semestre["tasa_homicidios"] = (
    agrupacion_semestre["homicidios"] / pob_total_2021
) * 100000

# Se calcula la tasa de homicidios del semestre inmediatamente anterior
agrupacion_semestre["tasa_semestre_pasado"] = (
    agrupacion_semestre["semestre_pasado"] / pob_total_2021
) * 100000

In [37]:
# Vemos las tasas de homicidios
agrupacion_semestre.head()

,anio,semestre,homicidios,semestre_pasado,tasa_homicidios,tasa_semestre_pasado
0,2016,1,65,0,2.111187,0.000000
1,2016,2,81,65,2.630864,2.111187
2,2017,1,69,81,2.241107,2.630864
3,2017,2,71,69,2.306066,2.241107
4,2018,1,70,71,2.273587,2.306066


### Se crea una columna con la diferencia porcentual semestre a semestre

In [38]:
agrupacion_semestre["dif_porcentual"] = (
    (
        agrupacion_semestre["tasa_homicidios"]
        - agrupacion_semestre["tasa_semestre_pasado"]
    )
    / agrupacion_semestre["tasa_semestre_pasado"]
    * 100
)

In [39]:
agrupacion_semestre.head()

,anio,semestre,homicidios,semestre_pasado,tasa_homicidios,tasa_semestre_pasado,dif_porcentual
0,2016,1,65,0,2.111187,0.000000,inf
1,2016,2,81,65,2.630864,2.111187,24.615385
2,2017,1,69,81,2.241107,2.630864,-14.814815
3,2017,2,71,69,2.306066,2.241107,2.898551
4,2018,1,70,71,2.273587,2.306066,-1.408451


### Se guarda el dataframe en un archivo CSV

In [40]:
agrupacion_semestre_csv = "data/01-kpi.csv"
agrupacion_semestre.to_csv(agrupacion_semestre_csv, index=False)

# KPI 2
### Reducir en un 7% la cantidad de accidentes mortales de motociclistas en el último año, en CABA, respecto al año anterior.

### Se filtra y selecciona registros donde la victima es motociclista

In [41]:
victima_moto = homicidios[(homicidios["victima"] == "MOTO")]

### Se agrupa por año y se realiza el conteo de homicidios

In [42]:
victima_moto = victima_moto.groupby(["anio"]).size().reset_index(name="homicidios")

### Se crea una nueva variable `anio_pasado` donde se obtiene la cantidad de homicidios ocurridos el año anterior 

In [43]:
victima_moto["anio_pasado"] = victima_moto["homicidios"].shift(periods=1, fill_value=0)

### Se obtiene la tasa de homicidios cada 100000 habitantes teniendo en cuenta que la población de Cuidad Autónoma de Buenos Aires al censo 2021

In [44]:
# Se obtiene la tasa de homicidios cada 100000 habitantes para el año actual
victima_moto["tasa_homicidios"] = (victima_moto["homicidios"] / pob_total_2021) * 100000

# Se obtiene la tasa de homicidios cada 100000 habitantes para el año anterior o pasado
victima_moto["tasa_anio_pasado"] = (
    victima_moto["anio_pasado"] / pob_total_2021
) * 100000

### Se obtiene la Variación Porcentual de los homicidios año a año

In [45]:
victima_moto["variacion"] = (
    (victima_moto["tasa_homicidios"] - victima_moto["tasa_anio_pasado"])
    / victima_moto["tasa_anio_pasado"]
) * 100


victima_moto["variacion"] = victima_moto["variacion"].replace([np.inf, -np.inf], np.nan)

victima_moto

,anio,homicidios,anio_pasado,tasa_homicidios,tasa_anio_pasado,variacion
0,2016,65,0,2.111187,0.000000,NaN
1,2017,56,65,1.818869,2.111187,-13.846154
2,2018,57,56,1.851349,1.818869,1.785714
3,2019,50,57,1.623990,1.851349,-12.280702
4,2020,28,50,0.909435,1.623990,-44.000000
5,2021,46,28,1.494071,0.909435,64.285714


### Se guarda el dataframe en un archivo CSV

In [46]:
victima_moto_csv = "data/02-kpi.csv"
victima_moto.to_csv(victima_moto_csv, index=False)